## Exercise Solution: Parallelization with Distributed

In [ ]:
using Distributed


In [ ]:

# Function to find prime numbers in a range using @distributed
function find_primes(start, stop)
    @everywhere function is_prime(n)
        if n <= 1
            return false
        end

        for i = 2:isqrt(n)
            if n % i == 0
                return false
            end
        end

        return true
    end

    results = @distributed (vcat) for n = start:stop
        if is_prime(n)
            [n]
        else
            Int[]
        end
    end

    return results
end



In [ ]:
# Set up the parallel workers
addprocs(4)

# Define the range to search for primes
start = 1
stop = 1000

# Perform the parallel prime number search
@time result = find_primes(start, stop)

# Print the result
println(result)

# Q: Is there a more elegant parallelized solution for the $\pi$ approximation?

In [ ]:
using Distributed
addprocs(4)

In [ ]:
"""
    calculate_pi_parallel_direct(total_points::Int) -> Float64

Calculate an estimate of π using the Monte Carlo method, in parallel, by directly distributing the random point generation across workers.

Instead of assigning a fixed number of points to each worker, this function parallelizes the task of generating and checking each point. Each iteration of the internal loop can be run on any available worker, which allows for improved load balancing.

# Arguments
- `total_points::Int`: The total number of random points to use for the estimation.

# Returns
- `Float64`: An estimate of π.
"""
function calculate_pi_parallel_direct(total_points::Int)
    # Use @distributed for parallel reduction with a (+) operator, iterating over each point
    inside_total = @distributed (+) for i = 1:total_points
        x = rand()
        y = rand()
        (x^2 + y^2) <= 1.0 ? 1 : 0
    end
    
    # Calculate pi using the aggregated result
    return 4 * inside_total / total_points
end

In [ ]:
calculate_pi_parallel_direct(Int(1e10))

---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2018-2024 [Point 8 GmbH](https://point-8.de)_